In [1]:
# ability to give a gift to a friend on instagram
# ability to create tweets-like feature on instagram
# create a marketplace for educators to sell their courses on instagram

In [2]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback

from IPython.display import display, Markdown
import wandb
from wandb.integration.langchain import WandbTracer
import os

In [3]:
string_template = """\
You are a tech product manager. You have to help the user create a Product Requirement Document based on the questions the user asks you. The user will ask you specific questions about each topic they want to be included in the PRD. 

Do not repeat the same information again and again. Answers to each question should be unique and not repetitive. By this I mean do not repeat any ideas or sentences. Do not copy statements and ideas from previous sections. Any ideas or examples should only be in accordance to the particular section.

Format your responses in Markdown mode with each topic being the ##Heading, and your answer being the content. Highlight important points in **bold**. Give the PRD a suitable #Title.

For reference, let us say there are 3 people - A, B, and C belonging to different age groups, professions, and geographies. A is a 20-year-old college student from India. B is a 40-year-old working professional from the US. C is a 60-year-old retired person from the UK.
If required, for that particular section, you can use any of these people as examples to explain your point.

You do not need to include these 3 people in every section. You can use them as examples only if required. You can also use other examples if you want to. You can also use yourself as an example if you want to.

Current conversation:
{history}
Human: {input}
AI: """

prompt = PromptTemplate(
    template=string_template,
    input_variables=["history", "input"],
)

prompts_list = [
    """Product Overview:
Define the Purpose and Scope of this product. It should include how different groups of users across ages, genders, and geographies can use this product. Include an overview of the product. Why should one use this product? Define the target audience and stakeholders in detail. Also, include the rationale behind having the particular group as the target audience. Explain the gap it is trying to fill as well - how it is different from and better than other similar products?""",
    """Product Objectives:
First, analyze whether the product objectives align with the company objectives. Think aloud. Explain your reasoning. Also, talk about why and how the business models of the product and company match. What company goals can the product help achieve - be it attracting customers, generating profits, or promoting the goodwill of the company? Also, explain how it would do this.""",
    """Market Research:
First, list out current and potential competitors. Current competitors should include already established businesses/products. Potential competitors should include products and businesses that aren’t yet popular or are still under development/ beta version. Also include major or minor differences between our product and the competitor products you have identified. Analyze how aspects of our product or competitor products are better for that particular aspect. How do the target customers different? Does our product better cater to current trends and expectations of the users? How? What should the product include to meet those trends and expectations.""",
    """Competitive Analysis Table:
Use all the above competitors to create a competitive analysis of these applications in a tabular form using the following points - user base, user region, different features supported, and pricing tiers. Don't limit yourself to these categories and think of other categories yourself. Return the output in a well-structured Markdown table""",
    """Feature Requirements:
What are some of the important features that should be implemented? Follow the MoSCoW format (Must have, Should have, Could have, Won’t have, along with why). How are we going to collect user inputs and use user data that we collect to make the product better and add other features?""",
    """Launch Strategy:
Compare US vs International markets for this product. Also, analyze this product and figure out what customer demographic is this product for. Based on these things, come up with a detailed launch strategy for the product. List the TAM vs SAM vs SOM. TAM or Total Available Market is the total market demand for a product or service. SAM or Serviceable Available Market is the segment of the TAM targeted by your products and services which is within your geographical reach. SOM or Serviceable Obtainable Market is the portion of SAM that you can capture.""",
    """User Stories:
Create user stories for the product. User stories are short, simple descriptions of a feature told from the perspective of the person who desires the new capability, usually a user or customer of the system. They typically follow a simple template: As a < type of user >, I want < some goal > so that < some reason >. For example, As a college student, I want to be able to share my notes with my friends so that I can help them with their studies.""",
    """Acceptance Criteria:
Define the quality of completeness required to be able to get to the MVP stage of this product.""",
    """Success Metrics:
How do we define success in this product? What are the KPIs to look out for? How are they measured? Why do those KPIs matter? How are we going to use these KPIs to make the product better?""",
    """Technical Feasibilities:
Outline the technical roadmap for this product. What mobile devices should this application be available for? What is a scalable and reliable tech stack which can be used for the frontend and the backend for this application?""",
    """Timeline: 
Define the timeline for the product development. In addition to the timeline, what are the resources required to complete this project. Think about the resources required for each stage of the project, the number of employees required for each stage, and the time required for each stage."""
]


In [4]:
chat = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    openai_api_key=os.environ["OPENAI_API_KEY"],
)

wandb.init(
    project="chat-prd-gpt-4",
    config={
        "model": "gpt-4",
        "temperature": 0
    },
    entity="arihantsheth",
    name="Instagram Threads_prd_gpt-4",
)

memory = ConversationBufferMemory()

chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=prompt,
    verbose=False
)

total_tokens = 0
total_cost = 0.0

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arihantsheth. Use `wandb login --relogin` to force relogin


In [5]:
with get_openai_callback() as callback:

    initial_output = chain.predict(
        input="""\
I want to add the following new feature:
Instagram Threads.

Feature description: A new app by Instagram that allows users to post tweet-like content.

DO NOT START WRITING. WAIT FOR THE HUMAN TO WRITE "Start generating the PRD" BEFORE YOU START WRITING.
"""
    )

    total_tokens += callback.total_tokens
    total_cost += callback.total_cost

display(Markdown(f"{initial_output}"))

Sure, I understand. Please let me know when you're ready for me to start generating the PRD.

In [6]:
output = ""
for i, prompt in enumerate(prompts_list):
    
    with get_openai_callback() as callback:

        output += chain.predict(
            input=prompt,
            callbacks=[WandbTracer()]
        )

        output += "\n\n"

wandb.log({"prd": output})
wandb.finish()

prd,# Product Requiremen...


In [7]:
output += chain.predict(
    input="Timeline: Define the timeline for the product development. In addition to the timeline, what are the resources required to complete this project. Think about the resources required for each stage of the project, the number of employees required for each stage, and the time required for each stage.",
    callbacks=[WandbTracer()]
)

output += "\n\n"

wandb: Streaming LangChain activity to W&B at https://wandb.ai/arihantsheth/productx/runs/mdniyd97
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


In [8]:
# with open("../generated_prds/Instagram Threads PRD Chat gpt-4.md", "w") as f:
#     f.write(output)

In [9]:
display(Markdown(output))

# Product Requirement Document for Instagram Threads

## Product Overview

Instagram Threads is a new feature designed to enhance the user experience by allowing them to post tweet-like content. This feature aims to bridge the gap between Instagram's primarily visual content and the text-based content popular on platforms like Twitter. 

The purpose of Instagram Threads is to provide a platform for users to share their thoughts, ideas, and opinions in a concise and engaging manner. It broadens the scope of Instagram by introducing a new way of communication, making it more versatile and appealing to a wider audience.

The target audience for Instagram Threads is vast and diverse. It includes but is not limited to:

- **Young adults and teenagers** who are the primary users of Instagram. For instance, A, a 20-year-old college student from India, can use Threads to share his thoughts on various topics, engage with his peers, and stay updated with the latest trends.
- **Working professionals** like B, a 40-year-old from the US, can use Threads to share industry insights, network with colleagues, and engage with thought leaders in their field.
- **Retired individuals** such as C, a 60-year-old from the UK, can use Threads to share their life experiences, engage with their community, and stay connected with family and friends.

The rationale behind targeting such a diverse audience is the universal appeal of sharing thoughts and ideas. Instagram Threads fills the gap in the market for a platform that combines the visual appeal of Instagram with the concise, text-based communication of Twitter. 

Unlike other similar products, Instagram Threads is integrated within the Instagram app, providing a seamless user experience. It leverages the existing user base and popularity of Instagram, making it a more convenient and attractive option for users. 

The stakeholders include Instagram users, advertisers, and Instagram as a company. Users benefit from a new way of communication, advertisers get a new platform to reach their audience, and Instagram can increase user engagement and ad revenue.

## Product Objectives

The product objectives of Instagram Threads align well with the overarching objectives of Instagram as a company. Instagram's mission is to bring people closer to the things they love, and Threads contributes to this mission by providing a new platform for users to express themselves and connect with others.

The business model of Instagram revolves around user engagement and advertising. Instagram Threads aligns with this model as it is designed to increase user engagement by providing a new way for users to interact. More engagement means more time spent on the app, which translates to more opportunities for advertisers to reach their target audience. 

Instagram Threads can help achieve several company goals:

- **Attracting new users**: By offering a unique feature that combines the visual appeal of Instagram with the text-based communication of Twitter, Instagram Threads can attract users who prefer text-based content, thereby expanding Instagram's user base.
- **Increasing user engagement**: By providing a new way for users to express themselves and interact with others, Threads can increase user engagement, leading to more time spent on the app.
- **Generating profits**: More user engagement means more ad impressions, which can lead to increased ad revenue. Additionally, Instagram could potentially introduce premium features within Threads, creating another revenue stream.
- **Promoting goodwill**: By continuously innovating and introducing new features like Threads, Instagram can strengthen its reputation as a user-centric platform that values its users' needs and preferences.

In conclusion, Instagram Threads aligns with Instagram's company objectives and business model, and it can contribute significantly to achieving the company's goals.

## Market Research

### Current Competitors:

1. **Twitter**: The primary competitor, as it already allows users to post tweet-like content. However, Instagram Threads offers a more visually appealing platform and is integrated within Instagram, providing a seamless user experience.

2. **Facebook**: Allows users to post text-based content, but Instagram Threads is more concise and engaging, catering to the trend of short, impactful content.

3. **Snapchat**: Offers a platform for sharing visual content and short text messages, but Instagram Threads allows for more in-depth text-based discussions.

### Potential Competitors:

1. **TikTok**: Currently focused on short video content, but could potentially expand into text-based content.

2. **LinkedIn**: Primarily a professional networking platform, but has been increasing its focus on content creation and could potentially introduce a similar feature.

The target customers for Instagram Threads are more diverse compared to its competitors. While Twitter and LinkedIn cater to a more mature audience, and Snapchat and TikTok cater to a younger demographic, Instagram Threads is designed to appeal to all age groups.

Instagram Threads is in line with the current trend of short, impactful content, similar to tweets or TikTok videos. It caters to the users' expectations of having a platform that allows for quick and easy sharing of thoughts and ideas.

To meet these trends and expectations, Instagram Threads should include features like:

- **Character limit**: To ensure the content remains concise and engaging.
- **Integration with Instagram**: To allow users to share their threads on their Instagram stories or feed.
- **Customizable privacy settings**: To give users control over who can see their threads.
- **Interactive features**: Such as polls or questions to increase user engagement.

## Competitive Analysis Table

| Competitor | User Base (in billions) | User Region | Different Features Supported | Pricing Tiers | Unique Selling Proposition |
|------------|-------------------------|-------------|------------------------------|---------------|----------------------------|
| Twitter    | 0.33                    | Global      | Tweeting, Trending, Retweeting, Direct Messaging | Free, Twitter Blue ($2.99/month) | Real-time updates and news |
| Facebook   | 2.8                     | Global      | Posting, Messaging, Marketplace, Groups, Events | Free, Ads | Comprehensive social networking |
| Snapchat   | 0.5                     | Primarily US and Europe | Snaps, Stories, Discover, Snap Map | Free, Ads | Ephemeral content and AR filters |
| TikTok     | 1                       | Global      | Short videos, Duet, Discover, Live Streaming | Free, Ads | Short, creative video content |
| LinkedIn   | 0.77                    | Global      | Networking, Job postings, Content sharing, Learning | Free, Premium (starting at $29.99/month) | Professional networking and career development |

## Feature Requirements

### Must Have:

- **Thread Creation**: Users must be able to create and post threads. This is the core functionality of Instagram Threads.
- **Integration with Instagram**: Threads should be shareable on Instagram stories or feed. This is crucial for leveraging the existing Instagram user base.
- **Privacy Settings**: Users must have control over who can see their threads. This is important for user privacy and security.

### Should Have:

- **Interactive Features**: Features like polls or questions should be included to increase user engagement.
- **Notifications**: Users should be notified when someone they follow posts a thread or when their thread receives engagement. This will keep users active and engaged.

### Could Have:

- **Thread Recommendations**: Based on user activity and interests, Instagram Threads could recommend threads for users to read. This could increase user engagement and time spent on the app.
- **Premium Features**: Instagram could introduce premium features within Threads, such as advanced customization options or analytics. This could create an additional revenue stream.

### Won't Have:

- **Long-form Content**: Instagram Threads won't support long-form content. The focus is on short, concise content to keep users engaged and maintain the fast-paced nature of the platform.

To make the product better and add other features, we will collect user inputs through surveys, user feedback, and analyzing user behavior within the app. This data will be used to understand what features users like, what they don't like, and what they want to see in the future. This will guide the development of new features and improvements to existing ones.

## Launch Strategy

### US vs International Markets:

The US market is a key target for Instagram Threads due to the high usage of social media platforms and the popularity of Instagram in the country. However, the international market also presents a significant opportunity, given the global reach of Instagram and the universal appeal of sharing thoughts and ideas.

### Customer Demographic:

Instagram Threads is designed for a broad demographic, including young adults, working professionals, and retired individuals. It caters to anyone who wants to share their thoughts and ideas in a concise and engaging manner.

### Launch Strategy:

1. **Beta Testing**: Start with a beta version of Instagram Threads, available to a select group of users. This will allow us to gather initial feedback and make necessary adjustments before the full launch.

2. **US Launch**: Launch Instagram Threads in the US market. Leverage Instagram's popularity and user base in the US to gain initial traction.

3. **International Rollout**: Gradually roll out Instagram Threads to other markets, starting with countries where Instagram is popular, such as India, Brazil, and the UK.

4. **Marketing Campaign**: Run a marketing campaign to promote Instagram Threads. This could include social media ads, influencer partnerships, and in-app promotions.

5. **Continuous Improvement**: Collect user feedback and data to continuously improve Instagram Threads and add new features.

### TAM vs SAM vs SOM:

- **TAM (Total Available Market)**: Given the global popularity of Instagram and the universal appeal of sharing thoughts and ideas, the TAM for Instagram Threads could be as high as the total number of social media users worldwide, which is estimated to be around 3.6 billion.

- **SAM (Serviceable Available Market)**: The SAM would be the number of Instagram users, as Instagram Threads is a feature within Instagram. As of 2020, Instagram has over 1 billion users worldwide.

- **SOM (Serviceable Obtainable Market)**: The SOM would be a fraction of the SAM, depending on the adoption rate of Instagram Threads. If we assume an adoption rate of 10%, the SOM would be around 100 million users.

## User Stories

1. **User Story 1**: As a college student, I want to share my thoughts on various topics with my peers on Instagram Threads, so that I can engage in meaningful discussions and stay updated with the latest trends.

2. **User Story 2**: As a working professional, I want to share industry insights on Instagram Threads, so that I can network with colleagues and engage with thought leaders in my field.

3. **User Story 3**: As a retired individual, I want to share my life experiences on Instagram Threads, so that I can engage with my community and stay connected with family and friends.

4. **User Story 4**: As an Instagram user, I want to control who can see my threads, so that I can maintain my privacy and security.

5. **User Story 5**: As an Instagram user, I want to be notified when someone I follow posts a thread or when my thread receives engagement, so that I can stay active and engaged on the platform.

6. **User Story 6**: As an Instagram user, I want to discover new threads based on my interests, so that I can find content that is relevant and interesting to me.

## Acceptance Criteria

To reach the Minimum Viable Product (MVP) stage for Instagram Threads, the following criteria must be met:

1. **Thread Creation**: Users should be able to create and post threads. The interface for creating threads should be user-friendly and intuitive.

2. **Integration with Instagram**: Threads should be shareable on Instagram stories or feed. The integration should be seamless, with no significant impact on the performance of the Instagram app.

3. **Privacy Settings**: Users should have control over who can see their threads. The privacy settings should be easy to understand and modify.

4. **Notifications**: Users should receive notifications when someone they follow posts a thread or when their thread receives engagement. The notifications should be timely and accurate.

5. **Stability and Performance**: The introduction of Instagram Threads should not negatively impact the stability or performance of the Instagram app. The app should remain fast and responsive, even with the added functionality.

6. **User Feedback**: Initial user feedback should be positive, indicating that users find Instagram Threads useful and enjoyable to use. Any significant issues or concerns raised by users should be addressed before moving beyond the MVP stage.

## Success Metrics

Success for Instagram Threads can be defined by the following Key Performance Indicators (KPIs):

1. **User Adoption Rate**: This measures the number of Instagram users who start using Threads. A high adoption rate indicates that users find the feature useful and engaging.

2. **User Engagement**: This includes metrics like the number of threads created, the number of replies to threads, and the time spent on Threads. High user engagement indicates that users are actively using and enjoying the feature.

3. **Retention Rate**: This measures the number of users who continue to use Threads over time. A high retention rate indicates that users find the feature valuable and continue to use it regularly.

4. **User Feedback**: This includes user reviews and ratings, as well as feedback collected through surveys or user interviews. Positive user feedback indicates that users are satisfied with the feature.

5. **Revenue Generation**: If premium features are introduced, revenue generated from these features can be a key success metric.

These KPIs matter because they provide insight into how well Instagram Threads is performing and how users are interacting with the feature. They can help identify areas where the feature is doing well and areas where improvement is needed.

We will use these KPIs to continuously improve Instagram Threads. For example, if the user adoption rate is low, we might need to improve our marketing efforts or make the feature more prominent within the Instagram app. If user engagement is low, we might need to add more interactive features or improve the user interface. By closely monitoring these KPIs and taking action based on the insights they provide, we can ensure that Instagram Threads is a success.

## Technical Feasibilities

### Technical Roadmap:

1. **Design Phase**: Design the user interface and user experience for Instagram Threads. This includes the thread creation interface, the thread viewing interface, and the integration with Instagram.

2. **Development Phase**: Develop the backend and frontend for Instagram Threads. This includes implementing the thread creation and viewing functionality, the privacy settings, the integration with Instagram, and the notification system.

3. **Testing Phase**: Conduct thorough testing to ensure that Instagram Threads works as expected and that there are no major bugs or issues. This includes functional testing, performance testing, and security testing.

4. **Launch Phase**: Launch Instagram Threads to a select group of users for beta testing. Collect feedback and make necessary adjustments.

5. **Full Launch**: Launch Instagram Threads to all Instagram users. Continue to monitor performance and collect user feedback.

### Mobile Devices:

Instagram Threads should be available for all mobile devices that support the Instagram app. This includes iOS devices (iPhone and iPad) and Android devices.

### Tech Stack:

A scalable and reliable tech stack for Instagram Threads could include:

- **Frontend**: React Native for building the user interface. This allows for cross-platform development for both iOS and Android devices.

- **Backend**: Node.js for the server-side logic. It's scalable and efficient, making it suitable for a high-traffic app like Instagram.

- **Database**: MongoDB for storing user data. It's a NoSQL database that can handle large amounts of data and is easy to scale.

- **Cloud Services**: AWS for hosting the app and storing data. It's reliable, scalable, and offers a wide range of services that can support the needs of Instagram Threads.

- **Notification Service**: Firebase Cloud Messaging for sending notifications to users. It's reliable and can handle a large volume of notifications.

This tech stack is just a suggestion and the actual tech stack should be decided based on the specific needs and constraints of the project.

## Timeline and Resources

### Design Phase (1 month):

- **Resources**: UI/UX designers, Product Manager
- **Employees**: 2-3 designers, 1 Product Manager

### Development Phase (3 months):

- **Resources**: Frontend and Backend Developers, Database Administrators, Product Manager
- **Employees**: 4-5 developers, 1-2 Database Administrators, 1 Product Manager

### Testing Phase (1 month):

- **Resources**: QA Engineers, Product Manager
- **Employees**: 2-3 QA Engineers, 1 Product Manager

### Launch Phase (1 month):

- **Resources**: Marketing team, Product Manager
- **Employees**: Marketing team size can vary based on the scale of the launch, 1 Product Manager

### Full Launch (Ongoing):

- **Resources**: Full team including developers, designers, QA engineers, marketing team, and Product Manager
- **Employees**: Team size can vary based on user feedback and the need for further development or adjustments

Please note that these timelines and resources are estimates and the actual requirements may vary based on the specific needs and constraints of the project.

